In [58]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
import spacy
from spacy.matcher import PhraseMatcher

# GOOGLE + FACEBOOK

In [59]:
def get_text_from_csv(fn): # get ad's text from csv file, return a dataframe with text, ad_id,ad_url, and report_url
    df = pd.read_csv(fn)
    df = df[df['ad_type'] == 'Text'].reset_index(drop = True) # only get text ads
    df_url = df[['ad_id','ad_url']].reset_index(drop = True) # get the urls
    urls = df_url['ad_url'].to_list() # create a list so we could get the report_urls
    report_urls = []
    for url in urls:
        entity_id = url.split('/')[-3]
        creative_id = url.split('/')[-1]
        report_url = 'https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id={}&creative_id={}&hl=en'.format(entity_id,creative_id)
        report_urls.append(report_url)
    df_report = pd.DataFrame(report_urls).rename(columns = {0:'report_url'})
    ad_text = [] # get texts from reports
    for report_url in report_urls:
        response = requests.get(report_url)
        text = response.text.split('"]')[0].split('[')[-1]
        ad_text.append(text)  
    df_text = pd.DataFrame(ad_text).rename(columns = {0:'text'})
    df_new = df[df['ad_type']=='Text'][['ad_id','ad_url']].reset_index(drop = True) 
    df = pd.concat([df_new,df_report,df_text],axis=1)
    return df  

In [60]:
# read google ads
google_df = get_text_from_csv('GoogleAds/CA.csv') 
google_df.head()

,ad_id,ad_url,report_url,text
0,CR221162606398799872,https://transparencyreport.google.com/political-ads/library/advertiser/AR206807382586556416/creative/CR221162606398799872,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR206807382586556416&creative_id=CR221162606398799872&hl=en,"""Ad Paid for by Dave Myers for San Diego County Sheriff 2022 | FPPC ID # 1436219"",""Don't Settle for the Status Quo! Elect Dave Myers for Sheriff come June 7th, 2022!"
1,CR436752647349862400,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR436752647349862400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR436752647349862400&hl=en,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."
2,CR149929646081703936,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR149929646081703936,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR149929646081703936&hl=en,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More."
3,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada."
4,CR370192611450814464,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR370192611450814464,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR370192611450814464&hl=en,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."


In [63]:
# read facebook ads
facebook_df = pd.read_csv('FacebookAds/california.csv')

In [64]:
# only keep text column
facebook_df.drop(columns = ['Unnamed: 0','byline'],inplace = True)
pd.set_option('display.max_colwidth', None)
facebook_df

,text
0,It is awesome to have the support of the California Nurses Association! What they have been through in this pandemic is unreal… If hospitals are going to continue not paying property taxes they must treat our nurses better!
1,"When the other candidate rides the coat tails of other establishment politicians, I ride in on my experience, dedication and support from the people…. Transparency and Credibility #ad5 #california"
2,"Seydi has worked with San Fernando Valley communities for 20 years. She understands the issues we are facing and she's prepared to advocate for us in Sacramento. \n\nVote for Seydi because she pledges toward fixing:\n\n📌 Homelessness\n📌 Housing\n📌 Education\n📌 Small Business\n📌 Access to healthcare\n\nMost importantly, she pledges to listen to her constituents. Vote for Seydi Alejandra Morales for State Senate District 20 starting May 9th!"
3,Learn more about our efforts to find solutions that will protect our planet while building a more equitable future for CA families.
4,Special interests like 𝗥𝗲𝗰𝗼𝗹𝗼𝗴𝘆 are exploiting environmental problems for their own profits. \n\nThe San Francisco-based garbage and recycling giant was fined $36 million in a public corruption scandal after it admitted to bribery allegations related to overcharging customers $100 million. Don’t let special interests make up for lost profits by raising taxes on California taxpayers.
...,...
10043,Help Protect Atkins Place!
10044,Help Protect Atkins Place!
10045,Help Protect Atkins Place!
10046,Help Protect Atkins Place!


In [151]:
# cncat google_df and facebook_df by 'text'
df_text = pd.concat([google_df,facebook_df])
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10198 entries, 0 to 10047
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       150 non-null    object
 1   ad_url      150 non-null    object
 2   report_url  150 non-null    object
 3   text        10196 non-null  object
dtypes: object(4)
memory usage: 398.4+ KB


In [152]:
df_text.head()

,ad_id,ad_url,report_url,text
0,CR221162606398799872,https://transparencyreport.google.com/political-ads/library/advertiser/AR206807382586556416/creative/CR221162606398799872,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR206807382586556416&creative_id=CR221162606398799872&hl=en,"""Ad Paid for by Dave Myers for San Diego County Sheriff 2022 | FPPC ID # 1436219"",""Don't Settle for the Status Quo! Elect Dave Myers for Sheriff come June 7th, 2022!"
1,CR436752647349862400,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR436752647349862400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR436752647349862400&hl=en,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."
2,CR149929646081703936,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR149929646081703936,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR149929646081703936&hl=en,"""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."",""Stop The Deceptive Corp Prop: Would Turn Every Screen Into A Gambling Device. Learn More."
3,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,"""Stand with Captain Sam Brown. Let's restore accountability and leadership to Washington."",""Captain Sam Brown fought for our Country. Now, he is ready to serve the people of Nevada."
4,CR370192611450814464,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR370192611450814464,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR370192611450814464&hl=en,"""A Broad Coalition of CA Indian Tribes Opposes This Online Gambling Prop. Learn More."",""Take Action: Corp Prop Would Legalize An Expansion of Online Sports Gambling in CA."


In [153]:
# check the dataset 
if df_text['text'].isnull().sum():
    print (df_text['text'].isnull().sum())
    df_text = df_text[df_text['text'].notnull()]
    df_text.reset_index(drop=True,inplace=True)

2


In [154]:
# function that filter the urls and symbols in the text 
def clean_text(df_text):
    def filter_text(x):
        url = 'http[s]?://\S+'
        x = re.sub(url,'',x)
        x = re.sub("[^\w\s]",' ',x) # filter symbols
        x = re.sub("\s+",' ',x)
        ls=[w.lower() for w in x.split()] 
        return ' '.join(ls)
    df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
    df_text['text'] = df_text['text'].replace('/u0026', ' ')
    df_text.drop_duplicates(subset = 'text',keep = 'first', inplace = True)
    df_text.reset_index(drop = True,inplace = True)
    return df_text

In [155]:
df_text = clean_text(df_text)

In [156]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ad_id       68 non-null     object
 1   ad_url      68 non-null     object
 2   report_url  68 non-null     object
 3   text        286 non-null    object
dtypes: object(4)
memory usage: 9.1+ KB


In [157]:
# import the Lexicon
with open ('lexicon.json') as f1:
    dic1 = json.load(f1)
# check the keys
dic1.keys()

dict_keys(['covid', 'economic', 'education', 'environment', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social and cultural', 'social programs', 'donation'])

In [158]:
# import en_core_web
#This only happens to Ying that she couldn't import 'en_core_web_sm' in Jupyter Notebook, so she imports it with the full path
nlp = spacy.load("/usr/local/Cellar/jupyterlab/3.2.9/libexec/lib/python3.9/site-packages/en_core_web_lg/en_core_web_lg-3.2.0/")

In [159]:
# the function that find the lexicon words in the text
def find_words(x,lexicon):
    topics= lexicon.keys()  
    doc = nlp(x) # nlp() is spaCy 2.2 English language model 
    words= []
    for t in topics:
        matcher = PhraseMatcher(nlp.vocab)
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", patterns) # spaCy2.2 phrase matcher
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')

In [160]:
# tagging the topic in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()    
    if x=='no words':
        return ''    
    if x != 'no words': 
        words = x.split(',')
        labels = []        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                labels.append(t)                
                #l = sorted(labels)        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [161]:
def get_word_lable(df_text):
    df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
    df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
    df_text['m_label'] = df_text['m_label'].apply(lambda x: 'no topic' if x=='' else x)
    return df_text

In [162]:
df_text = get_word_lable(df_text)
df_text.head()


,ad_id,ad_url,report_url,text,words,m_label
0,CR221162606398799872,https://transparencyreport.google.com/political-ads/library/advertiser/AR206807382586556416/creative/CR221162606398799872,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR206807382586556416&creative_id=CR221162606398799872&hl=en,ad paid for by dave myers for san diego county sheriff 2022 fppc id 1436219 don t settle for the status quo elect dave myers for sheriff come june 7th 2022,no words,no topic
1,CR436752647349862400,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR436752647349862400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR436752647349862400&hl=en,take action corp prop would legalize an expansion of online sports gambling in ca a broad coalition of ca indian tribes opposes this online gambling prop learn more,no words,no topic
2,CR149929646081703936,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR149929646081703936,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR149929646081703936&hl=en,take action corp prop would legalize an expansion of online sports gambling in ca stop the deceptive corp prop would turn every screen into a gambling device learn more,no words,no topic
3,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,no words,no topic
4,CR370192611450814464,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR370192611450814464,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR370192611450814464&hl=en,a broad coalition of ca indian tribes opposes this online gambling prop learn more take action corp prop would legalize an expansion of online sports gambling in ca,no words,no topic


In [163]:
df_text.to_csv('ads_ca.csv')

In [167]:
def count_topic(df_text):
    df_tag = df_text['m_label'].value_counts().rename_axis('topics').reset_index(name='counts')
    df_tag = df_tag.assign(single_topic=df_tag['topics'].str.split(',')).explode('single_topic').reset_index(drop = True)
    df_tag = df_tag.groupby('single_topic').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_tag.reset_index(drop = True, inplace = True)
    return df_tag

In [168]:
text_topic = count_topic(df_text)
text_topic

,single_topic,counts
0,no topic,138
1,economic,56
2,environment,36
3,donation,31
4,health,25
5,safety,19
6,social programs,18
7,social and cultural,17
8,education,15
9,covid,12


In [169]:
def count_word(df_text):
    df_words = df_text['words'].value_counts().rename_axis('words').reset_index(name='counts')
    df_words = df_words.assign(single_word=df_words['words'].str.split(',')).explode('single_word').reset_index(drop = True)
    df_words = df_words.groupby('single_word').sum().reset_index().sort_values(by = 'counts', ascending = False)
    df_words.reset_index(drop = True, inplace = True)
    return df_words

In [170]:
text_word = count_word(df_text)
text_word

,single_word,counts
0,no words,138
1,health,19
2,donate,18
3,tax,12
4,climate,12
...,...,...
167,greenhouse gas,1
168,greenhouse,1
169,genocide,1
170,gas emissions,1


# GOOGLE

In [171]:
if google_df['text'].isnull().sum():
    print (google_df['text'].isnull().sum())
    google_df = google_df[google_df['text'].notnull()]
    google_df.reset_index(drop=True,inplace=True)

In [172]:
google_df = clean_text(google_df)
google_df.info()

In [173]:
google_df = get_word_lable(google_df)
google_df.head()

,ad_id,ad_url,report_url,text,words,m_label
0,CR221162606398799872,https://transparencyreport.google.com/political-ads/library/advertiser/AR206807382586556416/creative/CR221162606398799872,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR206807382586556416&creative_id=CR221162606398799872&hl=en,ad paid for by dave myers for san diego county sheriff 2022 fppc id 1436219 don t settle for the status quo elect dave myers for sheriff come june 7th 2022,no words,no topic
1,CR436752647349862400,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR436752647349862400,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR436752647349862400&hl=en,take action corp prop would legalize an expansion of online sports gambling in ca a broad coalition of ca indian tribes opposes this online gambling prop learn more,no words,no topic
2,CR149929646081703936,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR149929646081703936,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR149929646081703936&hl=en,take action corp prop would legalize an expansion of online sports gambling in ca stop the deceptive corp prop would turn every screen into a gambling device learn more,no words,no topic
3,CR548953410917892096,https://transparencyreport.google.com/political-ads/library/advertiser/AR199061254449397760/creative/CR548953410917892096,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR199061254449397760&creative_id=CR548953410917892096&hl=en,stand with captain sam brown let s restore accountability and leadership to washington captain sam brown fought for our country now he is ready to serve the people of nevada,no words,no topic
4,CR370192611450814464,https://transparencyreport.google.com/political-ads/library/advertiser/AR440853413404606464/creative/CR370192611450814464,https://transparencyreport.google.com/transparencyreport/api/v3/politicalads/creatives/details?entity_id=AR440853413404606464&creative_id=CR370192611450814464&hl=en,a broad coalition of ca indian tribes opposes this online gambling prop learn more take action corp prop would legalize an expansion of online sports gambling in ca,no words,no topic


In [174]:
google_topic = count_topic(google_df)
google_topic

,single_topic,counts
0,no topic,37
1,donation,13
2,economic,9
3,governance,4
4,health,4
5,covid,3
6,social programs,2
7,education,1
8,immigration,1
9,safety,1


In [175]:
google_word = count_word(google_df)
google_word

,single_word,counts
0,no words,37
1,donate,11
2,chip in,4
3,health,4
4,filibuster,3
5,health care,2
6,covid,2
7,taxpayers,2
8,inflation,2
9,care costs,1


# FACEBOOK

In [176]:
if facebook_df['text'].isnull().sum():
    print (facebook_df['text'].isnull().sum())
    facebook_df = facebook_df[facebook_df['text'].notnull()]
    facebook_df.reset_index(drop=True,inplace=True)

2


In [177]:
facebook_df = clean_text(facebook_df)
facebook_df.info

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/197128158.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].astype(str).apply(lambda x: filter_text(x))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/197128158.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['text'] = df_text['text'].replace('/u0026', ' ')
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/197128158.py:12: SettingWithCopyWarning: 
A value is try

<bound method DataFrame.info of                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         text
0                                                                                                                                                                                                                                                                                               it is awesome to have the support of the california nurses association what they have been through in this pandemic is unreal if hospitals are going to continue not paying

In [178]:
facebook_df = get_word_lable(facebook_df)
facebook_df.head()

/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/1334858953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['words'] = df_text['text'].astype(str).apply(lambda x: find_words(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/1334858953.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['m_label'] = df_text['words'].apply(lambda x: find_topic(x,dic1))
/var/folders/fh/9s8xqf557hxb9jfsjxs123f80000gq/T/ipykernel_74898/1334858953.py:4: SettingWithCopyWa

,text,words,m_label
0,it is awesome to have the support of the california nurses association what they have been through in this pandemic is unreal if hospitals are going to continue not paying property taxes they must treat our nurses better,"taxes,pandemic","covid,economic"
1,when the other candidate rides the coat tails of other establishment politicians i ride in on my experience dedication and support from the people transparency and credibility ad5 california,no words,no topic
2,seydi has worked with san fernando valley communities for 20 years she understands the issues we are facing and she s prepared to advocate for us in sacramento vote for seydi because she pledges toward fixing homelessness housing education small business access to healthcare most importantly she pledges to listen to her constituents vote for seydi alejandra morales for state senate district 20 starting may 9th,"education,healthcare,small business","economic,education,health"
3,learn more about our efforts to find solutions that will protect our planet while building a more equitable future for ca families,no words,no topic
4,special interests like 𝗥𝗲𝗰𝗼𝗹𝗼𝗴𝘆 are exploiting environmental problems for their own profits the san francisco based garbage and recycling giant was fined 36 million in a public corruption scandal after it admitted to bribery allegations related to overcharging customers 100 million don t let special interests make up for lost profits by raising taxes on california taxpayers,"taxpayers,taxes",economic


In [179]:
facebook_topic = count_topic(facebook_df)
facebook_topic

,single_topic,counts
0,no topic,101
1,economic,47
2,environment,36
3,health,21
4,donation,18
5,safety,18
6,social and cultural,17
7,social programs,16
8,education,14
9,covid,9


In [181]:
facebook_word = count_word(facebook_df)
facebook_word

,single_word,counts
0,no words,101
1,health,15
2,climate,12
3,tax,11
4,school,8
...,...,...
154,greenhouse gas,1
155,greenhouse,1
156,genocide,1
157,gas emissions,1
